Pedersen Commitment
==================

## Pedersen commitment:

$Gen(1^\lambda) \rightarrow ck$

$$
g \leftarrow \mathbb{G}\\
h = g^x\\
ck:=(\mathbb{G}, p, g, h)
$$



$Com_{ck}(m) \rightarrow c$

$$
c:=(g^r, g^mh^r)
$$

In [1]:
from klefki.types.algebra.concrete import EllipticCurveCyclicSubgroupSecp256k1 as ECC
from klefki.types.algebra.concrete import EllipticCurveGroupSecp256k1 as Curve
from klefki.types.algebra.concrete import FiniteFieldCyclicSecp256k1 as CF
from klefki.types.algebra.concrete import FiniteFieldSecp256k1 as F
from klefki.types.algebra.utils import randfield

G = ECC.G
H = Curve.lift_x(F(1234567))


### $\Sigma$-protocol

Consider a commitment $A$ opening to m to be part of the statement. The prover computes a random commitment $B = Com_{ck}(m; s)$ and sends it to the veriﬁer, which answer with a random challenge $x$. The prover then sends opening information $z$ to the veriﬁer, which checks the commitment $A^x B$ opens to m using randomness $z$.

$s \leftarrow \mathbb{Z}_p$
$B=Com_{ck}(m;s)$
$$
P \rightarrow V: B
$$

In [2]:
m = randfield(CF)
r = randfield(CF)

A = G ** m + H ** r

In [3]:
s = randfield(CF)


B = G ** s * H ** r

$x \leftarrow \mathbb{Z}_p$

$$
P \leftarrow V: x
$$

In [4]:
e = randfield(CF)


$z = me + s; x = re + r$

$$
P \rightarrow V: z, x
$$

In [5]:
z = m*e + s
x = r*e + r

Accept $\iff$ $B \in \mathbf{G}, z \in \mathbb{Z}_p$

$$
Com_{zk}(z;x) = A^eB
$$

In [6]:
G ** z * H ** x == A ** e * B

True

## Implementation

In [7]:
from klefki.zkp.pedersen import PedersonCommitment

In [14]:
m = randfield(CF)
r = randfield(CF)

s = randfield(CF)
e = randfield(CF)
P = PedersonCommitment(G, G@x)

In [9]:
P.commit(m, s, r)

<bound method PedersonCommitment.commit of <klefki.zkp.pedersen.PedersonCommitment object at 0x1031f2358>>

In [10]:
P.callange(x)

(FiniteFieldCyclicSecp256k1::61073159043296165203183829394239089189239291086668539547484419164272797761860,
 FiniteFieldCyclicSecp256k1::42020470230205876527880542820709302653026446003532590861100457256970823006480)

In [11]:
P.proof()

True

In [12]:
m1 = randfield(CF)


P.trapdoor(m1, x)

In [13]:
P.callange(e)
P.proof()

True